---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [83]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    spam = len(spam_data[spam_data['target']==1])
    total = spam_data.shape[0]
    return (spam/total)*100

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    cv = CountVectorizer().fit(X_train)
    len_lst = [len(x) for x in cv.get_feature_names()]
    
    return cv.get_feature_names()[np.argmax(len_lst)]

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [224]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    cv = CountVectorizer().fit(X_train)
    X_train_vectorized = cv.transform(X_train)
    X_test_vectorized = cv.transform(X_test)
    
    model = MultinomialNB(alpha=0.1).fit(X_train_vectorized,y_train)
    prediction = model.predict(X_test_vectorized)
    
    return roc_auc_score(y_test,prediction)

In [225]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
len(X_train)

4179

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect_tfidf = TfidfVectorizer().fit(X_train)
X_train_vectorized = vect_tfidf.transform(X_train)

In [11]:
X_train_vectorized

<4179x7354 sparse matrix of type '<class 'numpy.float64'>'
	with 55130 stored elements in Compressed Sparse Row format>

In [12]:
doc = 6
feature_index = X_train_vectorized[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [X_train_vectorized[doc, x] for x in feature_index])
list(tfidf_scores)

[(7305, 0.06632067425788671),
 (7042, 0.14330016183230146),
 (6588, 0.37747796780025566),
 (6587, 0.17282609864122322),
 (6407, 0.2059448631921737),
 (6143, 0.26739572828955477),
 (5658, 0.1329629555898702),
 (5400, 0.2163855662539368),
 (4407, 0.09458014299826566),
 (4276, 0.2530568446352961),
 (4218, 0.2530568446352961),
 (3348, 0.2530568446352961),
 (2928, 0.2530568446352961),
 (2887, 0.22623488920411522),
 (2782, 0.09387164742520636),
 (2472, 0.17378593282458957),
 (1929, 0.17477830989058163),
 (1811, 0.1588328817490513),
 (1768, 0.3965297050210666),
 (1541, 0.10377192307738273),
 (1500, 0.13369786414477738),
 (491, 0.16536481116829066)]

In [47]:
nonzero = X_train_vectorized.nonzero()
nonzero #첫번째가 row(문장) , 두번째가 column(단어)

(array([   0,    0,    0, ..., 4178, 4178, 4178], dtype=int32),
 array([7305, 7085, 6421, ..., 2065,  931,  790], dtype=int32))

In [48]:
nonzero_tfidf = [X_train_vectorized[nonzero[0][idx],nonzero[1][idx]] for idx in range(len(nonzero[0]))]
nonzero_tfidf_df = pd.DataFrame(nonzero_tfidf,index=nonzero[1],columns=['tfidf'])
nonzero_tfidf_df = nonzero_tfidf_df.reset_index()
def changeIdxWord(item):
    return vect_tfidf.get_feature_names()[item]
nonzero_tfidf_df

,index,tfidf
0,7305,0.198151
1,7085,0.352010
2,6421,0.390988
3,4624,0.471125
4,3933,0.354634
...,...,...
55125,3336,0.179845
55126,3199,0.275090
55127,2065,0.347443
55128,931,0.241310


In [49]:
vect_tfidf.get_feature_names()

['00',
 '000',
 '000pes',
 '008704050406',
 '0089',
 '0121',
 '01223585236',
 '01223585334',
 '0125698789',
 '02',
 '0207',
 '02072069400',
 '02073162414',
 '02085076972',
 '021',
 '03',
 '04',
 '0430',
 '05',
 '0578',
 '06',
 '07008009200',
 '07099833605',
 '07123456789',
 '0721072',
 '07734396839',
 '07742676969',
 '07753741225',
 '0776xxxxxxx',
 '07781482378',
 '07786200117',
 '077xxx',
 '078',
 '07808247860',
 '07808726822',
 '07821230901',
 '078498',
 '07880867867',
 '0789xxxxxxx',
 '07946746291',
 '0796xxxxxx',
 '07973788240',
 '07xxxxxxxxx',
 '08',
 '0800',
 '08000407165',
 '08000839402',
 '08000930705',
 '08000938767',
 '08001950382',
 '08002888812',
 '08002986030',
 '08002986906',
 '08006344447',
 '0808',
 '08081263000',
 '08081560665',
 '0825',
 '0844',
 '08448350055',
 '08448714184',
 '0845',
 '08452810071',
 '08452810073',
 '08452810075over18',
 '0870',
 '08700435505150p',
 '08700469649',
 '08700621170150p',
 '08701213186',
 '08701237397',
 '08701417012',
 '08701417012150p'

In [50]:
tfidf_max = nonzero_tfidf_df.groupby('index').max()
tfidf_max = tfidf_max.reset_index()
tfidf_max['word'] = [vect_tfidf.get_feature_names()[idx] for idx in tfidf_max['index']]
tfidf_max

,index,tfidf,word
0,0,0.237314,00
1,1,0.361385,000
2,2,0.228624,000pes
3,3,0.260735,008704050406
4,4,0.242052,0089
...,...,...,...
7349,7349,0.566275,û_thanks
7350,7350,0.284009,ûï
7351,7351,0.278142,ûïharry
7352,7352,0.362962,ûò


In [51]:
m = tfidf_max.sort_values(['tfidf','word'],ascending=[False,True]).iloc[:20]

In [55]:
m

,index,tfidf,word
274,274,1.000000,146tf150p
533,533,1.000000,645
931,931,1.000000,anything
933,933,1.000000,anytime
1214,1214,1.000000,beerage
2298,2298,1.000000,done
2496,2496,1.000000,er
3180,3180,1.000000,havent
3293,3293,1.000000,home
3848,3848,1.000000,lei


In [56]:
pd.Series(list(m['tfidf']),index=m['word'])

word
146tf150p    1.000000
645          1.000000
anything     1.000000
anytime      1.000000
beerage      1.000000
done         1.000000
er           1.000000
havent       1.000000
home         1.000000
lei          1.000000
nite         1.000000
ok           1.000000
okie         1.000000
thank        1.000000
thanx        1.000000
too          1.000000
where        1.000000
yup          1.000000
tick         0.980166
blank        0.932702
dtype: float64

In [36]:
tfidf_min = nonzero_tfidf_df.groupby('index').min()
tfidf_min = tfidf_min.reset_index()
tfidf_min['word'] = [vect_tfidf.get_feature_names()[idx] for idx in tfidf_max['index']]
tfidf_min

,index,tfidf,word
0,0,0.184719,00
1,1,0.193701,000
2,2,0.228624,000pes
3,3,0.251494,008704050406
4,4,0.242052,0089
...,...,...,...
7349,7349,0.566275,û_thanks
7350,7350,0.284009,ûï
7351,7351,0.278142,ûïharry
7352,7352,0.270956,ûò


In [61]:
tfidf_min.sort_values(['tfidf','word'],ascending=[True,True]).iloc[:20]

,index,tfidf,word
6561,6561,0.023205,to
3558,3558,0.027758,it
6446,6446,0.028577,the
4564,4564,0.031338,not
3548,3548,0.031698,is
1044,1044,0.031934,at
1193,1193,0.032058,be
3445,3445,0.032877,in
3336,3336,0.034464,how
849,849,0.034802,all


In [232]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect_tfidf = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect_tfidf.transform(X_train)
    
    nonzero = X_train_vectorized.nonzero()
    nonzero_tfidf = [X_train_vectorized[nonzero[0][idx],nonzero[1][idx]] for idx in range(len(nonzero[0]))]
    nonzero_tfidf_df = pd.DataFrame(nonzero_tfidf,index=nonzero[1],columns=['tfidf'])
    nonzero_tfidf_df = nonzero_tfidf_df.reset_index()
    
    tfidf_max = nonzero_tfidf_df.groupby('index').max()
    tfidf_max = tfidf_max.reset_index()
    tfidf_max['word'] = [vect_tfidf.get_feature_names()[idx] for idx in tfidf_max['index']]
    tfidf_max = tfidf_max.sort_values(['tfidf','word'],ascending=[False,True]).iloc[:20]
    
    tfidf_min = nonzero_tfidf_df.groupby('index').max()
    tfidf_min = tfidf_min.reset_index()
    tfidf_min['word'] = [vect_tfidf.get_feature_names()[idx] for idx in tfidf_min['index']]
    tfidf_min = tfidf_min.sort_values(['tfidf','word'],ascending=[True,True]).iloc[:20]
    
    result_max = pd.Series(list(tfidf_max['tfidf']),index=tfidf_max['word'])
    result_min = pd.Series(list(tfidf_min['tfidf']),index=tfidf_min['word'])
    
    return (result_min,result_max)

In [233]:
answer_four()

(word
 aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 afternoons      0.091250
 approaching     0.091250
 dtype: float64,
 word
 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [69]:
vect_tfidf = TfidfVectorizer(min_df=3).fit(X_train)
X_train_vectorized = vect_tfidf.transform(X_train)
vect_tfidf.get_feature_names()
vect_tfidf.vocabulary_

{'ll': 1164,
 'text': 1947,
 'you': 2281,
 'when': 2191,
 'drop': 656,
 'off': 1400,
 'hi': 940,
 'mate': 1236,
 'its': 1046,
 'did': 606,
 'hav': 918,
 'nice': 1364,
 'just': 1077,
 'message': 1264,
 'say': 1685,
 'hello': 934,
 'coz': 539,
 'sent': 1718,
 'in': 1006,
 'ages': 172,
 'started': 1850,
 'driving': 655,
 'so': 1800,
 'stay': 1855,
 'network': 1355,
 'operator': 1425,
 'the': 1958,
 'service': 1723,
 'is': 1037,
 'free': 800,
 'for': 790,
 'visit': 2121,
 '80488': 114,
 'biz': 332,
 'activate': 152,
 'your': 2282,
 '500': 95,
 'messages': 1265,
 'by': 393,
 'replying': 1630,
 'to': 2003,
 'this': 1973,
 'with': 2218,
 'word': 2234,
 'terms': 1944,
 'conditions': 509,
 'www': 2253,
 'com': 492,
 'girls': 853,
 'are': 233,
 'waiting': 2141,
 'them': 1961,
 'now': 1385,
 'great': 879,
 'night': 1367,
 'chatting': 452,
 'send': 1715,
 'stop': 1863,
 'how': 973,
 'much': 1330,
 'an': 205,
 'can': 414,
 'further': 825,
 'club': 481,
 'tones': 2014,
 'see': 1703,
 'my': 1338,
 't

In [78]:
def answer_five():
    vect_tfidf = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect_tfidf.transform(X_train)
    X_test_vectorized = vect_tfidf.transform(X_test)
    
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized,y_train)
    prediction = model.predict(X_test_vectorized)
    
    return roc_auc_score(y_test,prediction)

In [79]:
answer_five()

0.9905660377358491

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [86]:
spam_data['len'] = [len(doc) for doc in spam_data['text']]
spam_data.groupby('target').mean()

,len
target,
0,71.023627
1,138.866131


In [87]:
def answer_six():
    spam_data['len'] = [len(doc) for doc in spam_data['text']]
    grouped = spam_data.groupby('target').mean()
    return (grouped.loc[0,'len'],grouped.loc[1,'len'])

In [88]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [110]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [94]:
vect_tfidf = TfidfVectorizer(min_df=5).fit(X_train)
X_train_vectorized = vect_tfidf.transform(X_train)
X_test_vectorized = vect_tfidf.transform(X_test)
X_train_vectorized

<4179x1468 sparse matrix of type '<class 'numpy.float64'>'
	with 46095 stored elements in Compressed Sparse Row format>

In [115]:
X_train.shape

(4179,)

In [117]:
doc_len = [len(doc) for doc in X_train]
doc_len

[31,
 130,
 66,
 146,
 124,
 23,
 128,
 35,
 32,
 25,
 24,
 22,
 160,
 28,
 28,
 133,
 125,
 27,
 35,
 153,
 162,
 64,
 139,
 54,
 28,
 128,
 30,
 46,
 55,
 33,
 38,
 64,
 26,
 45,
 34,
 104,
 50,
 154,
 30,
 51,
 24,
 39,
 28,
 25,
 81,
 145,
 50,
 29,
 17,
 36,
 32,
 122,
 64,
 32,
 148,
 43,
 408,
 47,
 160,
 22,
 49,
 168,
 13,
 29,
 36,
 74,
 17,
 32,
 74,
 157,
 114,
 155,
 35,
 155,
 27,
 35,
 43,
 109,
 149,
 162,
 109,
 79,
 47,
 77,
 35,
 70,
 51,
 152,
 143,
 141,
 55,
 27,
 101,
 39,
 39,
 52,
 65,
 40,
 35,
 38,
 29,
 149,
 156,
 32,
 147,
 38,
 35,
 81,
 95,
 64,
 109,
 156,
 23,
 23,
 152,
 67,
 22,
 121,
 169,
 158,
 104,
 119,
 23,
 24,
 28,
 163,
 86,
 34,
 72,
 77,
 81,
 141,
 44,
 50,
 39,
 45,
 155,
 97,
 37,
 106,
 30,
 34,
 13,
 33,
 158,
 27,
 127,
 39,
 32,
 69,
 74,
 33,
 26,
 62,
 126,
 169,
 34,
 111,
 120,
 27,
 96,
 24,
 27,
 56,
 36,
 26,
 107,
 148,
 53,
 145,
 104,
 69,
 72,
 24,
 136,
 158,
 49,
 44,
 163,
 130,
 134,
 38,
 26,
 67,
 82,
 53,
 49,
 36,

In [118]:
add_feature(X_train_vectorized,doc_len)

<4179x1469 sparse matrix of type '<class 'numpy.float64'>'
	with 50274 stored elements in Compressed Sparse Row format>

In [226]:
from sklearn.svm import SVC

def answer_seven():
    def add_feature(X, feature_to_add):
        """
        Returns sparse feature matrix with added feature.
        feature_to_add can also be a list of features.
        """
        from scipy.sparse import csr_matrix, hstack
        return hstack([X, csr_matrix(feature_to_add).T], 'csr')
    
    vect_tfidf = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect_tfidf.transform(X_train)
    X_test_vectorized = vect_tfidf.transform(X_test)    
    X_train_vectorized = add_feature(X_train_vectorized,[len(doc) for doc in X_train])
    X_test_vectorized = add_feature(X_test_vectorized,[len(doc) for doc in X_test])
    
    model = SVC(C=10000).fit(X_train_vectorized,y_train)
    prediction = model.predict(X_test_vectorized)
    return roc_auc_score(y_test,prediction)

In [227]:
answer_seven()

0.9661689557407943

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [139]:
def getNumDigit(string):
    return sum(c.isdigit() for c in string)

spam_data['num_digit'] = [getNumDigit(doc) for doc in spam_data['text']]
spam_data.groupby('target').mean()

,len,num_digit
target,,
0,71.023627,0.299275
1,138.866131,15.759036


In [142]:
def answer_eight():
    spam_data['num_digit'] = [sum(c.isdigit() for c in doc) for doc in spam_data['text']]
    grouped = spam_data.groupby('target').mean()
    return (grouped.loc[0,'num_digit'],grouped.loc[1,'num_digit'])

In [143]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [160]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    def add_feature(X, feature_to_add):
        """
        Returns sparse feature matrix with added feature.
        feature_to_add can also be a list of features.
        """
        from scipy.sparse import csr_matrix, hstack
        return hstack([X, csr_matrix(feature_to_add).T], 'csr')
    
    vect_tfidf = TfidfVectorizer(min_df=5,ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect_tfidf.transform(X_train)
    X_test_vectorized = vect_tfidf.transform(X_test)    
    X_train_vectorized = add_feature(X_train_vectorized,[len(doc) for doc in X_train])
    X_train_vectorized = add_feature(X_train_vectorized,[sum(c.isdigit() for c in doc) for doc in X_train])
    X_test_vectorized = add_feature(X_test_vectorized,[len(doc) for doc in X_test])    
    X_test_vectorized = add_feature(X_test_vectorized,[sum(c.isdigit() for c in doc) for doc in X_test])
    
    model = LogisticRegression(C=100).fit(X_train_vectorized,y_train)
    #model = LogisticRegression(C=100,max_iter=2000).fit(X_train_vectorized,y_train)
    prediction = model.predict(X_test_vectorized)
    
    return roc_auc_score(y_test,prediction)

In [161]:
answer_nine()

C:\Users\netis\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9884028364593043

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [163]:
import re
text = 'hello %%#@this is'
len(re.findall(r'\W',text))

6

In [167]:
spam_data['num_non_word'] = [len(re.findall(r'\W',doc)) for doc in spam_data['text']]
grouped = spam_data.groupby('target').mean()
grouped

,len,num_digit,num_non_word
target,,,
0,71.023627,0.299275,17.291813
1,138.866131,15.759036,29.041499


In [166]:
spam_data

,text,target,len,num_digit,num_non_word
0,"Go until jurong point, crazy.. Available only ...",0,111,0,28
1,Ok lar... Joking wif u oni...,0,29,0,11
2,Free entry in 2 a wkly comp to win FA Cup fina...,1,155,25,33
3,U dun say so early hor... U c already then say...,0,49,0,16
4,"Nah I don't think he goes to usf, he lives aro...",0,61,0,14
...,...,...,...,...,...
5567,This is the 2nd time we have tried 2 contact u...,1,161,21,38
5568,Will Ì_ b going to esplanade fr home?,0,37,0,8
5569,"Pity, * was in mood for that. So...any other s...",0,57,0,16
5570,The guy did some bitching but I acted like i'd...,0,125,0,26


In [168]:
def answer_ten():
    import re
    spam_data['num_non_word'] = [len(re.findall(r'\W',doc)) for doc in spam_data['text']]
    grouped = spam_data.groupby('target').mean()
    
    return (grouped.loc[0,'num_non_word'],grouped.loc[1,'num_non_word'])

In [169]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [193]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

vect_tfidf = CountVectorizer(min_df=5,ngram_range=(2,5),analyzer='char_wb').fit(X_train)
X_train_vectorized = vect_tfidf.transform(X_train)
X_test_vectorized = vect_tfidf.transform(X_test) 

df_train = pd.DataFrame([[len(doc) for doc in X_train],
         [sum(c.isdigit() for c in doc) for doc in X_train],
         [len(re.findall(r'\W',doc)) for doc in X_train]],index = ['length_of_doc', 'digit_count', 'non_word_char_count'])

df_test = pd.DataFrame([[len(doc) for doc in X_test],
     [sum(c.isdigit() for c in doc) for doc in X_test],
     [len(re.findall(r'\W',doc)) for doc in X_test]],index = ['length_of_doc', 'digit_count', 'non_word_char_count'])

X_train_vectorized = add_feature(X_train_vectorized,df_train)
X_test_vectorized = add_feature(X_test_vectorized,df_test)

model = LogisticRegression(C=100,max_iter=2000).fit(X_train_vectorized,y_train)
prediction = model.predict(X_test_vectorized)


In [208]:
coeff = model.coef_[0]
coeff.sort()
coeff

array([-0.62264827, -0.60799874, -0.54677537, ...,  0.57898698,
        0.61905366,  1.47659575])

In [214]:
coeff[:10]

array([-0.62264827, -0.60799874, -0.54677537, -0.54618682, -0.51165218,
       -0.50735158, -0.50427451, -0.48591303, -0.47257103, -0.46728126])

In [215]:
coeff[:-10:-1]

array([1.47659575, 0.61905366, 0.57898698, 0.5777927 , 0.57013544,
       0.56465334, 0.54183617, 0.54094161, 0.53615428])

In [171]:
X_train_vectorized

<4179x16316 sparse matrix of type '<class 'numpy.int64'>'
	with 746763 stored elements in Compressed Sparse Row format>

In [172]:
X_train.shape
# (3,4179)

(4179,)

In [190]:
df = pd.DataFrame([[len(doc) for doc in X_train],
             [sum(c.isdigit() for c in doc) for doc in X_train],
             [len(re.findall(r'\W',doc)) for doc in X_train]],index = ['length_of_doc', 'digit_count', 'non_word_char_count'])

X_train_vectorized = add_feature(X_train_vectorized, df)

In [216]:
def answer_eleven():
    import re
    
    def add_feature(X, feature_to_add):
        """
        Returns sparse feature matrix with added feature.
        feature_to_add can also be a list of features.
        """
        from scipy.sparse import csr_matrix, hstack
        return hstack([X, csr_matrix(feature_to_add).T], 'csr')
    
    vect_tfidf = CountVectorizer(min_df=5,ngram_range=(2,5),analyzer='char_wb').fit(X_train)
    X_train_vectorized = vect_tfidf.transform(X_train)
    X_test_vectorized = vect_tfidf.transform(X_test) 
    
    df_train = pd.DataFrame([[len(doc) for doc in X_train],
             [sum(c.isdigit() for c in doc) for doc in X_train],
             [len(re.findall(r'\W',doc)) for doc in X_train]],index = ['length_of_doc', 'digit_count', 'non_word_char_count'])
    
    df_test = pd.DataFrame([[len(doc) for doc in X_test],
         [sum(c.isdigit() for c in doc) for doc in X_test],
         [len(re.findall(r'\W',doc)) for doc in X_test]],index = ['length_of_doc', 'digit_count', 'non_word_char_count'])

    X_train_vectorized = add_feature(X_train_vectorized,df_train)
    X_test_vectorized = add_feature(X_test_vectorized,df_test)
    
    model = LogisticRegression(C=100).fit(X_train_vectorized,y_train)
    prediction = model.predict(X_test_vectorized)
    
    coeff = model.coef_[0]
    coeff.sort()
    
    return (roc_auc_score(prediction,y_test),coeff[:10],coeff[:-10:-1])

In [217]:
answer_eleven()

C:\Users\netis\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.9918774285317791,
 array([-1.53230334, -1.29749374, -0.91735616, -0.78474499, -0.77132418,
        -0.72739922, -0.70493751, -0.69269104, -0.6864433 , -0.67967923]),
 array([1.57438201, 0.8085238 , 0.80723617, 0.77828125, 0.66791756,
        0.66490494, 0.65877183, 0.63931452, 0.62672943]))